In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor


ImportError: cannot import name 'r2_squared' from 'sklearn.metrics' (c:\Users\prana\anaconda3\envs\FirstML\Lib\site-packages\sklearn\metrics\__init__.py)

In [2]:
from xgboost import XGBRegressor

In [3]:
df = pd.read_csv('D:\ML Projects\First project\Research\stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
x = df.drop(['writing_score'], axis= 1)

In [6]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score
0,female,group B,bachelor's degree,standard,none,72,72
1,female,group C,some college,standard,completed,69,90
2,female,group B,master's degree,standard,none,90,95
3,male,group A,associate's degree,free/reduced,none,47,57
4,male,group C,some college,standard,none,76,78


In [7]:
y = df['writing_score']

In [20]:
y.head()

0    74
1    88
2    93
3    44
4    75
Name: writing_score, dtype: int64

In [21]:
y.shape

(1000,)

In [9]:
num_feat = x.select_dtypes(exclude="object").columns
cat_feat = x.select_dtypes(include='object').columns

num_feat, cat_feat

(Index(['math_score', 'reading_score'], dtype='object'),
 Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
        'test_preparation_course'],
       dtype='object'))

In [10]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [11]:
onehot = OneHotEncoder()
std_sclar = StandardScaler()

transformer = ColumnTransformer([
    ("OneHotEncoder",onehot,cat_feat),
    ("StandardScaler",std_sclar,num_feat)
])

In [12]:
x.shape

(1000, 7)

In [13]:
x = transformer.fit_transform(x)

In [15]:
x.shape

(1000, 19)

In [17]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.39002351,  0.19399858],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.19207553,  1.42747598],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.57771141,  1.77010859],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -0.46775108,  0.12547206],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12609287,  0.60515772],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.71993682,  1.15336989]])

In [29]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=10)
x_train.shape, x_test.shape

((700, 19), (300, 19))

In [38]:
def validation(actual, pred):
    return (mean_absolute_error(actual, pred), np.sqrt(mean_squared_error(actual, pred)), r2_score(actual, pred))

In [33]:
models = {
    'LinearReg': LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(),
    "Adaboost": AdaBoostRegressor()
}

In [37]:
list(models)

['LinearReg',
 'Ridge',
 'Lasso',
 'DecisionTree',
 'RandomForest',
 'XGBoost',
 'Adaboost']

In [40]:
mode_list = []
r2_scores = []

for i in list(models):
    model = models[i]
    model.fit(x_train,y_train)
    
    train_pred = model.predict(x_train)
    test_pred = model.predict(x_test)
    
    mae, mse, r2 = validation(y_train, train_pred)
    print("Model: {}: Evaluation on Train data: mae: {}, mse: {}, r2: {}".format(i,mae,mse,r2))
    
    mae, mse, r2 = validation(y_test, test_pred)
    print("Model: {}: Evaluation on Test data: mae: {}, mse: {}, r2: {}".format(i,mae,mse,r2))
    mode_list.append(i)
    r2_scores.append(r2)

Model: LinearReg: Evaluation on Train data: mae: 2.8262081473214287, mse: 3.484657256224376, r2: 0.9483545655046908
Model: LinearReg: Evaluation on Test data: mae: 2.8163321940104167, mse: 3.4839302811548944, r2: 0.9449063832402268
Model: Ridge: Evaluation on Train data: mae: 2.825345337444263, mse: 3.484684977637065, r2: 0.9483537437939786
Model: Ridge: Evaluation on Test data: mae: 2.8147917729092233, mse: 3.481926360679434, r2: 0.9449697435639939
Model: Lasso: Evaluation on Train data: mae: 3.773760260288974, mse: 4.715986508734068, r2: 0.905407455685144
Model: Lasso: Evaluation on Test data: mae: 3.4753602541152744, mse: 4.251551351821212, r2: 0.9179540343871557
Model: DecisionTree: Evaluation on Train data: mae: 0.011428571428571429, mse: 0.15583874449479593, r2: 0.9998967089528062
Model: DecisionTree: Evaluation on Test data: mae: 4.293333333333333, mse: 5.215521706087193, r2: 0.876531012419977
Model: RandomForest: Evaluation on Train data: mae: 1.268800833333333, mse: 1.55820779

In [41]:
mode_list, r2_scores

(['LinearReg',
  'Ridge',
  'Lasso',
  'DecisionTree',
  'RandomForest',
  'XGBoost',
  'Adaboost'],
 [0.9449063832402268,
  0.9449697435639939,
  0.9179540343871557,
  0.876531012419977,
  0.9309315013166612,
  0.9228690234482833,
  0.9239411442354751])

In [42]:
pd.DataFrame(list(zip(mode_list, r2_scores)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
1,Ridge,0.944970
0,LinearReg,0.944906
4,RandomForest,0.930932
6,Adaboost,0.923941
5,XGBoost,0.922869
2,Lasso,0.917954
3,DecisionTree,0.876531
